# Constraints

- Time limit: integer 0<= T <= 999
- Number of bunnies: integer 0? <= B <= 5
- 


In [115]:
class Graph:
    def __init__(self, matrix):
        self.cost_matrix = matrix
        self.n = len(matrix)
        self.n_rabbits = self.n - 2
        self.edges = [(i, j, self.cost_matrix[i][j]) for i in range(self.n) for j in range(self.n)]

    def any_self_distance_negative(self):
        for i in range(self.n):
            if self.cost_matrix[i][i] < 0:
                return True
        return False

    # Bellman-Ford algorithm
    def BellmanFord(self, start_node):

        # This implementation takes in a graph, represented as
        # lists of vertices (represented as integers [0..n-1]) and edges,
        # and fills two arrays (distance and predecessor) holding
        # the shortest path from the source to each vertex

        distance = [float("inf")] * self.n
        distance[start_node] = 0 # Even if the path to go to itself is not zero, you can just stay put
        
        # Step 2: relax edges repeatedly

        for _ in range(len(self.edges) - 1):
            for (u, v, weight) in self.edges:
                if distance[u] + weight < distance[v]:
                    distance[v] = distance[u] + weight

        # Step 3: check for negative-weight cycles
        for (u, v, weight) in self.edges:
            if distance[u] + weight < distance[v]:
                return "Negative cycle"

        return [int(d) for d in distance]

In [117]:
# depth first search
def dfs(graph, distances, path_list, cumulative_time_to_path, cost_limit):
    """
    Returns a list of paths that are valid starting from the given path with the given cumulative cost, and that end in the last node
    """
    print("Entrado en dfs con path", path_list, "y coste acumulado", cumulative_time_to_path)
    
    # if cumulative_time_to_path + distances[path_list[-1]][-1] > cost_limit:
    #     print("Retorno vacío!!! - No debería pasar------------------------------------------------------------------")
    #     return []
    
    if path_list[-1] == graph.n - 1:
        ret = path_list
        print("Retorno path en salida temprana", ret)
        return ret
    
    list_possible_nodes = []
    
    for possible_node in range(graph.n):
        if possible_node not in path_list and cumulative_time_to_path + distances[path_list[-1]][possible_node] + distances[possible_node][-1] <= cost_limit:
            list_possible_nodes.append(possible_node)
    
    print("Lista de posible nodos: ", list_possible_nodes)
    
    # We precluded this from happening already
    # if len(list_possible_nodes) == 0:
    #     print("Devuelve solo el camino dado (nada mas posible) - FALLO ---------------------------------------------------", path_list)
    #     return path_list
    
    possible_paths = [path_list + [possible_node] for possible_node in list_possible_nodes]
    possible_cumulative_times = [cumulative_time_to_path + distances[path_list[-1]][possible_node] for possible_node in list_possible_nodes]

    print("Posibles caminos en lista", possible_paths)
    print("Posibles tiempos en lista", possible_cumulative_times)
    ret = [dfs(graph, distances, path, cum_path_cost, cost_limit) for path, cum_path_cost in zip(possible_paths, possible_cumulative_times)]
    print("Retorno de dfs", ret)
    for path_list in ret:
        yield path_list

SyntaxError: 'return' with argument inside generator (<ipython-input-117-a96d9614d61c>, line 38)

In [118]:
def solve(graph, time_limit):
    if graph.any_self_distance_negative():
        # all vertices can be reached as long as you go back in time long enough by just staying put
        return [i for i in range(graph.n_rabbits)]
    
    distances = [graph.BellmanFord(i) for i in range(graph.n)] # distances[i][j] is the distance from i to j, minimised. Unless there is a negative cycle, so there's a "Negative cycle" string instead

    print("Bellmand-Ford distances", distances)
    if any([d == "Negative cycle" for d in distances]):
        return [i for i in range(graph.n_rabbits)]


    cumulative_time_of_path = 0
    path = [0]
    paths = dfs(graph, distances, path, cumulative_time_of_path, time_limit)
    print("paths = ", paths)
    # Eliminate paths that are not valid
    paths = [p for p in paths if p[-1] == graph.n - 1]
    print("Eliminated empty paths = ", paths)
    
    def score(path):
        """Counts the number of rabbits that are visited. Assumes that no rabbit is visited twice"""
        return sum([1 for i in range(1, graph.n - 1) if i in path])


    paths_score = [score(path) for path in paths]
    max_score_paths = [path for path, score in zip(paths, paths_score) if score == max(paths_score)]
    return max_score_paths[0]


    # for possible_node in range(graph.n):
    #     # possible_node = distances.index(min(distances))
    #     current_node = path[-1]
    #     distance_to_possible = distances[path[-1]][possible_node]
    #     distace_possible_to_end = distances[possible_node][-1]

    #     if cumulative_time_of_path + distance_to_possible + distace_possible_to_end <= time_limit:
    #         path.append(possible_node)
    #         cumulative_time_of_path += distance_to_possible
    #         yield possible_paths()
    #     else:
    #         yield path

    # # paths = possible_paths()
    # if paths == []:
    #     return None
    # else:
    #     return max(paths, key= lambda v: (1 in v) + (2 in v) + (3 in v) + (4 in v)  + (5 in v))
    #     # no tener en cuenta el último nodo, que es el de la meta

In [119]:
def solution(times, time_limit):
    graph = Graph(times)
    return solve(graph, time_limit)
    

In [120]:
# testing
solution([[0, 2, 2, 2, -1], [9, 0, 2, 2, -1], [9, 3, 0, 2, -1], [9, 3, 2, 0, -1], [9, 3, 2, 2, 0]], 1)

('Bellmand-Ford distances', [[0, 2, 1, 1, -1], [8, 0, 1, 1, -1], [8, 2, 0, 1, -1], [8, 2, 1, 0, -1], [9, 3, 2, 2, 0]])
('Entrado en dfs con path', [0], 'y coste acumulado', 0)
('Lista de posible nodos: ', [1, 2, 3, 4])
('Posibles caminos en lista', [[0, 1], [0, 2], [0, 3], [0, 4]])
('Posibles tiempos en lista', [2, 1, 1, -1])
('Entrado en dfs con path', [0, 1], 'y coste acumulado', 2)
('Lista de posible nodos: ', [4])
('Posibles caminos en lista', [[0, 1, 4]])
('Posibles tiempos en lista', [1])
('Entrado en dfs con path', [0, 1, 4], 'y coste acumulado', 1)
('Retorno path en salida temprana', [0, 1, 4])
('Retorno de dfs', [[0, 1, 4]])
('Entrado en dfs con path', [0, 2], 'y coste acumulado', 1)
('Lista de posible nodos: ', [3, 4])
('Posibles caminos en lista', [[0, 2, 3], [0, 2, 4]])
('Posibles tiempos en lista', [2, 0])
('Entrado en dfs con path', [0, 2, 3], 'y coste acumulado', 2)
('Lista de posible nodos: ', [4])
('Posibles caminos en lista', [[0, 2, 3, 4]])
('Posibles tiempos en list

[0, 4]